# Player Analysis Scrape

This scrapes all of NBA.com's individual player stats, including advanced stats.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\player_analysis_streamlit')
import warnings
warnings.filterwarnings('ignore')
import streamlit as st
from selenium.common.exceptions import WebDriverException
import winsound

In [16]:
data_folder = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\data\\player\\nba_com_playerdata\\'

In [17]:
sound_file = "data/audio/Jobs Done.wav"

def play_jobs_done():
    winsound.PlaySound(sound_file, winsound.SND_FILENAME)

In [18]:
# # get player numbers from file
# player_numbers = pd.read_csv(data_folder + 'player/nba_com_info/players_and_photo_links.csv')
# player_numbers

In [19]:
# # Source and destination paths -- just the first copy over
# src_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\data\\player\\nba_com_playerdata\\'
# dst_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\player_analysis_streamlit\\data\\player\\nba_com_playerdata\\'

# # Copy the folders recursively from source to destination
# shutil.copytree(src_dir, dst_dir)


MUST RUN THIS TO UPDATE (AFTER DOING REGULAR DAILY UPDATE)

In [20]:
# This updates it instead
# Source and destination paths
src_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\data\\player\\nba_com_playerdata\\'
dst_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\player_analysis_streamlit\\data\\player\\nba_com_playerdata\\'

# Copy files from source to destination
for root, dirs, files in os.walk(src_dir):
    # Determine the destination directory
    dst_root = os.path.join(dst_dir, os.path.relpath(root, src_dir))
    # Create the destination directory if it does not exist
    os.makedirs(dst_root, exist_ok=True)
    # Copy the files to the destination directory
    for file in files:
        src_file = os.path.join(root, file)
        dst_file = os.path.join(dst_root, file)
        shutil.copy2(src_file, dst_file)

In [21]:
# copy ''C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\data/player/aggregates/Trad&Adv_box_scores_GameView.csv to data/player/aggregates/Trad&Adv_box_scores_GameView.csv
src_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1\\data\\player\\aggregates\\'
dst_dir = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\player_analysis_streamlit\\data\\player\\aggregates\\'

# Copy files from source to destination
for root, dirs, files in os.walk(src_dir):
    # Determine the destination directory
    dst_root = os.path.join(dst_dir, os.path.relpath(root, src_dir))
    # Create the destination directory if it does not exist
    os.makedirs(dst_root, exist_ok=True)
    # Copy the files to the destination directory
    for file in files:
        if 'Trad&Adv' in file:
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dst_root, file)
            shutil.copy2(src_file, dst_file)

In [22]:
# stop everything from running here
play_jobs_done()

In [ ]:
# add capitalized player name
player_numbers['Player'] = player_numbers['player_name'].str.title()

Get splits url. You can get other splits urls by replacing traditional with:
- advanced
- scoring
- usage
- shooting

In [ ]:
# player_numbers['splits_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/traditional'
# player_numbers['advanced_splits_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/advanced'
# player_numbers['shooting_splits_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/shooting'
# player_numbers['misc_splits_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/misc'
# player_numbers['usage_splits_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/usage'

We should already have advanced box scores in the database, but you can also choose to download them at https://www.nba.com/stats/player/NBA_ID/boxscores-traditional

Add Matchups Data

In [ ]:
player_numbers['tracking_ShotDash_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/shots-dash'
# reb dash
player_numbers['tracking_RebDash_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/rebounds-dash'
# pass dash
player_numbers['tracking_PassDash_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/passes-dash'
# def dash
player_numbers['tracking_DefDash_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/defense-dash'

Add Matchups data

- by position on offense
- by position on defense
- Head to Head (defense)

In [ ]:
# head to head
player_numbers['head_to_head_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/head-to-head'
# head to head offense
player_numbers['head_to_head_off_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/head-to-head?Matchup=Offense'
# by position on defense
player_numbers['by_position_def_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/by-position'
# by position on offense
player_numbers['by_position_off_url'] = 'https://nba.com/stats/player/' + player_numbers['nba_id'].astype(str) + '/by-position?Matchup=offense'


In [ ]:
# get list of players in player_numbers
player_list = player_numbers['Player'].tolist()
player_list

['Precious Achiuwa',
 'Steven Adams',
 'Bam Adebayo',
 'Ochai Agbaji',
 'Santi Aldama',
 'Nickeil Alexander Walker',
 'Grayson Allen',
 'Jarrett Allen',
 'Jose Alvarado',
 'Kyle Anderson',
 'Giannis Antetokounmpo',
 'Thanasis Antetokounmpo',
 'Cole Anthony',
 'Og Anunoby',
 'Ryan Arcidiacono',
 'Deni Avdija',
 'Deandre Ayton',
 'Udoka Azubuike',
 'Ibou Badji',
 'Marvin Bagley Iii',
 'Patrick Baldwin Jr',
 'Lamelo Ball',
 'Lonzo Ball',
 'Mo Bamba',
 'Paolo Banchero',
 'Desmond Bane',
 'Dalano Banton',
 'Dominick Barlow',
 'Harrison Barnes',
 'Scottie Barnes',
 'Rj Barrett',
 'Will Barton',
 'Charles Bassey',
 'Keita Bates Diop',
 'Nicolas Batum',
 'Darius Bazley',
 'Bradley Beal',
 'Malik Beasley',
 'Marjon Beauchamp',
 'Davis Bertans',
 'Patrick Beverley',
 'Saddiq Bey',
 'Khem Birch',
 'Goga Bitadze',
 'Bismack Biyombo',
 'Buddy Boeheim',
 'Bogdan Bogdanovic',
 'Bojan Bogdanovic',
 'Bol Bol',
 'Leandro Bolmaro',
 'Devin Booker',
 'Brandon Boston Jr',
 'Chris Boucher',
 'James Bouknigh

In [ ]:
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')

To get all matchups, un-hash the code in the cell below.

In [ ]:
# all_matchups = pd.DataFrame()

# driver = None
# if driver == None:
#     driver = webdriver.Chrome()
#     # minimize
#     driver.minimize_window()


# for player in player_list:
#     chosen_player = player_numbers[player_numbers['Player'] == player]
#     # Get Urls for test
#     # Head to Head Matchups
#     head_to_head_url = chosen_player['head_to_head_url'].values[0]
#     head_to_head_off_url = chosen_player['head_to_head_off_url'].values[0]
#     by_position_def_url = chosen_player['by_position_def_url'].values[0]
#     by_position_off_url = chosen_player['by_position_off_url'].values[0]


#     try:
#         driver.get(head_to_head_url)
#         # wait 2 seconds
#         time.sleep(5)
#         # select all pages
#         all_pages_xpath = '//*[@id="__next"]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]'
#         all_pages = driver.find_element_by_xpath(all_pages_xpath)
#         all_pages.click()
#         # wait 5 seconds
#         time.sleep(5)

#         # click 'partial poss' to sort by partial possession
#         partial_pos_xpath = '//*[@id="__next"]/div[2]/div[2]/section/div[4]/section[3]/div/div[2]/div[3]/table/thead/tr/th[4]'
#         partial_pos = driver.find_element_by_xpath(partial_pos_xpath)
#         partial_pos.click()
#         # wait 2 seconds
#         time.sleep(2)

#         # find table
#         soup = BeautifulSoup(driver.page_source, 'html.parser')
#         table = soup.find_all('table')
#         # read table with pandas
#         player_d_matchups = pd.read_html(str(table))[0]
#         player_d_matchups['Player'] = player
#         player_d_matchups.to_csv('data/player/nba_com_playerdata/matchups/' + player + '_matchups_'+ today + '.csv', index=False)

#         # append to all_matchups
#         all_matchups = all_matchups.append(player_d_matchups)
#         print(player + ' matchups done')

#     except:
#         print(player + '...That didnt work')
#         pass

# all_matchups
        

In [ ]:
# # save to csv
# all_matchups.to_csv('data/player/nba_com_playerdata/matchups/all_matchups_' + today + '.csv', index=False)

In [ ]:
tracking_catch_shoot = 'https://www.nba.com/stats/players/catch-shoot'
tracking_drives = 'https://www.nba.com/stats/players/drives'
tracking_defensive_impact = 'https://www.nba.com/stats/players/defensive-impact'
tracking_passing = 'https://www.nba.com/stats/players/passing'
tracking_touches = 'https://www.nba.com/stats/players/touches'
tracking_pull_up_shots = 'https://www.nba.com/stats/players/pullup'
tracking_rebounds = 'https://www.nba.com/stats/players/rebounding'
tracking_shooting_efficiency = 'https://www.nba.com/stats/players/shooting-efficiency'
tracking_speed_distance = 'https://www.nba.com/stats/players/speed-distance'
tracking_elbow_touches = 'https://www.nba.com/stats/players/elbow-touch'
tracking_post_ups = 'https://www.nba.com/stats/players/tracking-post-ups'
tracking_paint_touches = 'https://www.nba.com/stats/players/paint-touch'

In [ ]:
driver = None
if driver == None:
    driver = webdriver.Chrome()
    # minimize
    driver.minimize_window()

In [ ]:
def download_player_tracking(url):
    try:
        driver.get(url)
        # wait 2 seconds
        time.sleep(8)
        # select all pages
        all_pages_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]'
        all_pages = driver.find_element_by_xpath(all_pages_xpath)
        all_pages.click()
        # wait 5 seconds
        time.sleep(5)
        # find table
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find_all('table')
        # read table with pandas
        df = pd.read_html(str(table))[2]
        return df
    except:
        print('That didnt work')
        pass

In [ ]:
catch_shoot = download_player_tracking(tracking_catch_shoot)
catch_shoot

,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,eFG%
0,A.J. Lawson,DAL,12,4.2,1.3,0.4,1.0,41.7,0.4,1.0,41.7,62.5
1,AJ Green,MIL,25,8.9,2.9,1.0,2.1,45.3,1.0,2.1,45.3,67.9
2,AJ Griffin,ATL,54,20.5,3.5,1.2,3.1,38.5,1.1,3.0,37.3,56.2
3,Aaron Gordon,DEN,49,30.3,1.9,0.7,1.8,36.8,0.6,1.5,37.3,52.9
4,Aaron Holiday,ATL,49,14.6,1.5,0.5,1.2,43.1,0.5,1.2,43.1,64.7
...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,47,20.9,2.3,0.9,2.1,39.6,0.6,1.5,37.5,53.0
501,Zach LaVine,CHI,53,36.0,4.5,1.5,3.5,44.3,1.5,3.3,44.6,65.8
502,Zeke Nnaji,DEN,44,13.3,0.9,0.3,1.0,30.4,0.3,1.0,29.5,44.6
503,Ziaire Williams,MEM,28,17.4,1.6,0.5,2.5,21.4,0.5,2.4,20.9,31.4


In [ ]:
drives = download_player_tracking(tracking_drives)
drives

,PLAYER,TEAM,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,...,PTS,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%
0,A.J. Lawson,DAL,12,5,7,4.2,0.1,0.0,0.0,0.0,...,0.0,0.0,0.1,100.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AJ Green,MIL,25,19,6,8.9,0.2,0.0,0.0,100.0,...,0.1,33.3,0.2,66.7,0.0,0.0,0.0,16.7,0.0,0.0
2,AJ Griffin,ATL,54,25,29,20.5,2.9,0.9,1.8,49.5,...,2.1,70.9,0.7,24.7,0.1,4.4,0.2,6.3,0.2,6.3
3,Aaron Gordon,DEN,49,35,14,30.3,3.3,0.9,1.5,58.3,...,2.5,76.5,0.9,25.9,0.3,8.0,0.3,9.3,0.6,17.3
4,Aaron Holiday,ATL,49,26,23,14.6,3.4,0.6,1.3,42.9,...,1.4,40.4,1.6,47.6,0.3,7.8,0.2,6.6,0.1,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,1.0,0.4,0.6,62.1,...,0.9,97.7,0.1,13.6,0.0,4.5,0.0,4.5,0.1,11.4
500,Zach LaVine,CHI,52,24,28,36.0,10.1,2.4,4.7,52.5,...,7.2,70.9,3.5,34.3,0.7,7.2,0.7,6.7,1.2,11.8
501,Zeke Nnaji,DEN,43,29,14,13.5,0.2,0.0,0.0,50.0,...,0.0,28.6,0.1,57.1,0.0,0.0,0.0,0.0,0.0,0.0
502,Ziaire Williams,MEM,28,16,12,17.4,1.5,0.3,0.7,45.0,...,0.8,51.2,0.5,36.6,0.1,7.3,0.2,12.2,0.0,2.4


In [ ]:
defensive_impact = download_player_tracking(tracking_defensive_impact)
if defensive_impact.empty:
    defensive_impact = download_player_tracking(tracking_defensive_impact)
defensive_impact

,Player,Team,GP,MIN,W,L,STL,BLK,DREB,DFGM,DFGA,DFG%
0,A.J. Lawson,DAL,12,4.2,5,7,0.0,0.0,0.6,0.3,0.3,100.0
1,AJ Green,MIL,25,8.9,19,6,0.2,0.0,0.9,0.8,1.0,79.2
2,AJ Griffin,ATL,54,20.5,25,29,0.7,0.1,1.6,1.6,2.1,77.2
3,Aaron Gordon,DEN,49,30.3,35,14,0.8,0.8,4.4,1.9,3.2,60.0
4,Aaron Holiday,ATL,49,14.6,26,23,0.6,0.2,0.9,0.8,1.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,21.1,13,33,0.4,0.7,4.0,3.2,5.8,54.5
500,Zach LaVine,CHI,52,36.0,24,28,1.0,0.2,4.3,1.6,2.1,77.8
501,Zeke Nnaji,DEN,43,13.5,29,14,0.3,0.4,1.1,1.4,2.2,63.4
502,Ziaire Williams,MEM,28,17.4,16,12,0.4,0.2,2.1,1.1,1.8,64.0


In [ ]:
passing = download_player_tracking(tracking_passing)
if passing.empty:
    passing = download_player_tracking(tracking_passing)
passing

,PLAYER,TEAM,GP,W,L,MIN,Passes Made,Passes Received,AST,Secondary AST,Potential AST,AST PTS Created,AST PTS Created.1,AST Adj,AST To Pass%,AST To Pass% Adj
0,A.J. Lawson,DAL,12,5,7,4.2,2.3,2.9,0.1,0.0,0.3,0.3,0.1,3.7,3.7,NaN
1,AJ Green,MIL,25,19,6,8.9,6.8,7.9,0.4,0.1,1.0,1.3,0.6,6.5,8.8,NaN
2,AJ Griffin,ATL,54,25,29,20.5,13.1,15.3,1.0,0.1,1.8,2.6,1.2,7.8,9.0,NaN
3,Aaron Gordon,DEN,49,35,14,30.3,26.6,27.9,2.9,0.4,4.2,7.1,3.4,10.8,12.9,NaN
4,Aaron Holiday,ATL,49,26,23,14.6,15.8,16.8,1.3,0.1,2.9,3.6,1.7,8.4,10.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,23.9,22.7,2.6,0.1,3.9,6.6,3.0,10.8,12.4,NaN
500,Zach LaVine,CHI,52,24,28,36.0,38.2,48.5,4.1,0.8,7.8,10.1,5.2,10.6,13.7,NaN
501,Zeke Nnaji,DEN,43,29,14,13.5,8.1,8.0,0.3,0.0,0.6,0.7,0.3,3.2,3.7,NaN
502,Ziaire Williams,MEM,28,16,12,17.4,11.9,12.9,0.9,0.1,1.6,2.4,1.1,7.2,9.6,NaN


In [ ]:
touches = download_player_tracking(tracking_touches)
touches

,Player,Team,GP,W,L,MIN,PTS,TOUCHES,Front CT Touches,Time Of Poss,Avg Sec Per Touch,Avg Drib Per Touch,PTS Per Touch,Elbow Touches,Post Ups,Paint Touches,PTS Per Elbow Touch,PTS Per Post Touch,PTS Per Paint Touch
0,A.J. Lawson,DAL,12,5,7,4.2,2.0,4.1,2.8,0.1,1.42,0.63,0.490,0.0,0.0,0.1,0.000,0.000,2.000
1,AJ Green,MIL,25,19,6,8.9,4.2,10.9,6.6,0.5,2.52,1.75,0.390,0.1,0.0,0.4,0.000,0.000,0.889
2,AJ Griffin,ATL,54,25,29,20.5,9.3,22.1,15.1,0.8,2.05,1.47,0.421,0.5,0.0,0.6,0.852,2.000,0.879
3,Aaron Gordon,DEN,49,35,14,30.3,17.3,42.8,23.7,2.1,2.95,2.12,0.405,1.8,2.3,7.0,0.478,0.757,1.185
4,Aaron Holiday,ATL,49,26,23,14.6,4.3,21.2,8.6,1.4,3.99,3.61,0.203,0.1,0.0,0.4,0.286,0.000,0.571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,10.1,34.2,23.4,1.0,1.83,0.63,0.295,3.5,2.8,4.9,0.589,0.767,0.853
500,Zach LaVine,CHI,52,24,28,36.0,23.8,62.8,32.8,4.4,4.17,3.41,0.378,0.5,0.2,0.9,0.643,0.250,1.292
501,Zeke Nnaji,DEN,43,29,14,13.5,5.2,12.5,9.0,0.3,1.51,0.38,0.414,1.1,0.4,3.0,0.417,0.944,1.086
502,Ziaire Williams,MEM,28,16,12,17.4,6.0,19.0,11.9,0.7,2.09,1.45,0.313,0.2,0.0,1.3,1.000,0.000,1.371


In [ ]:
pull_up_shots = download_player_tracking(tracking_pull_up_shots)
pull_up_shots

,PLAYER,TEAM,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,eFG%
0,A.J. Lawson,DAL,12,5,7,4.2,0.0,0.0,0.2,0.0,0.0,0.1,0.0,0.0
1,AJ Green,MIL,25,19,6,8.9,1.0,0.4,1.0,38.5,0.2,0.6,25.0,46.2
2,AJ Griffin,ATL,54,25,29,20.5,3.1,1.4,2.7,50.7,0.4,0.8,47.7,58.0
3,Aaron Gordon,DEN,49,35,14,30.3,1.8,0.7,1.7,39.8,0.4,0.9,45.7,52.4
4,Aaron Holiday,ATL,49,26,23,14.6,0.9,0.4,1.0,39.2,0.1,0.3,37.5,45.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,47,13,34,20.9,0.1,0.0,0.2,18.2,0.0,0.1,0.0,18.2
501,Zach LaVine,CHI,53,25,28,36.0,6.6,2.7,7.5,36.1,1.2,3.8,31.2,44.1
502,Zeke Nnaji,DEN,44,29,15,13.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,Ziaire Williams,MEM,28,16,12,17.4,1.4,0.6,1.3,45.7,0.2,0.5,46.2,54.3


In [ ]:
rebounds = download_player_tracking(tracking_rebounds)
rebounds

,PLAYER,TEAM,GP,W,L,MIN,REB,Contested REB,Contested REB%,REB Chances,REB Chance%,Deferred REB Chances,Adjusted REB Chance%,AVG REB Distance
0,A.J. Lawson,DAL,12,5,7,4.2,0.7,0.3,50.0,1.2,57.1,0.2,66.7,5.5
1,AJ Green,MIL,25,19,6,8.9,1.0,0.4,34.6,2.1,50.0,0.2,54.2,8.1
2,AJ Griffin,ATL,54,25,29,20.5,2.1,0.5,24.1,3.8,55.2,0.2,58.3,8.9
3,Aaron Gordon,DEN,49,35,14,30.3,6.9,3.1,45.8,11.0,62.3,0.6,65.9,5.8
4,Aaron Holiday,ATL,49,26,23,14.6,1.3,0.3,21.9,2.7,47.8,0.2,51.6,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,5.8,2.8,48.3,12.2,47.6,0.9,51.2,4.8
500,Zach LaVine,CHI,52,24,28,36.0,4.8,0.8,16.5,8.8,54.1,0.7,59.0,7.4
501,Zeke Nnaji,DEN,43,29,14,13.5,2.4,1.2,49.0,5.4,44.2,0.4,47.7,5.6
502,Ziaire Williams,MEM,28,16,12,17.4,2.5,0.6,24.6,4.4,55.6,0.2,58.5,7.0


In [ ]:
shooting_efficiency = download_player_tracking(tracking_shooting_efficiency)
shooting_efficiency

,PLAYER,TEAM,GP,W,L,MIN,PTS,Drive PTS,Drive FG%,C&S PTS,C&S FG%,Pull Up PTS,Pull Up FG%,Paint Touch PTS,Paint Touch FG%,Post Touch PTS,Post Touch FG%,Elbow Touch PTS,Elbow Touch FG%,eFG%
0,A.J. Lawson,DAL,12,5,7,4.2,2.0,0.0,0.0,1.3,41.7,0.0,0.0,0.2,100.0,0.0,0.0,0.0,0.0,57.5
1,AJ Green,MIL,25,19,6,8.9,4.2,0.1,100.0,2.9,44.4,1.0,38.5,0.3,75.0,0.0,0.0,0.0,0.0,60.7
2,AJ Griffin,ATL,54,25,29,20.5,9.3,2.1,49.5,3.6,38.6,3.1,50.7,0.5,70.0,0.0,100.0,0.4,73.3,57.5
3,Aaron Gordon,DEN,49,35,14,30.3,17.3,2.5,58.3,2.0,35.8,1.8,39.8,8.2,75.0,1.7,50.9,0.9,58.3,63.3
4,Aaron Holiday,ATL,49,26,23,14.6,4.3,1.4,42.9,1.5,43.1,0.9,39.2,0.2,41.7,0.0,0.0,0.0,25.0,50.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,10.1,0.9,62.1,2.6,40.2,0.1,18.2,4.2,68.1,2.2,56.3,2.1,70.8,59.1
500,Zach LaVine,CHI,52,24,28,36.0,23.8,7.2,52.5,4.7,43.2,6.7,36.1,1.2,78.9,0.0,0.0,0.3,46.7,54.6
501,Zeke Nnaji,DEN,43,29,14,13.5,5.2,0.0,50.0,1.0,30.4,0.0,0.0,3.2,67.4,0.4,63.6,0.5,72.7,61.5
502,Ziaire Williams,MEM,28,16,12,17.4,6.0,0.8,45.0,1.6,21.1,1.4,45.7,1.7,76.9,0.0,0.0,0.2,33.3,48.4


In [ ]:
speed_distance = download_player_tracking(tracking_speed_distance)
speed_distance

,PLAYER,TEAM,GP,W,L,MIN,Dist. Feet,Dist. Miles,Dist. Miles Off,Dist. Miles Def,Avg Speed,Avg Speed Off,Avg Speed Def
0,A.J. Lawson,DAL,12,5,7,4.2,1794.6,0.34,0.18,0.16,4.48,4.64,4.33
1,AJ Green,MIL,25,19,6,8.8,3644.0,0.69,0.38,0.31,4.36,4.76,3.90
2,AJ Griffin,ATL,53,25,28,20.6,8821.0,1.67,0.90,0.77,4.54,4.92,4.17
3,Aaron Gordon,DEN,49,35,14,30.3,11395.5,2.16,1.13,1.03,4.01,4.24,3.79
4,Aaron Holiday,ATL,49,26,23,14.7,5938.4,1.12,0.57,0.55,4.28,4.37,4.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,45,13,32,20.9,8040.8,1.52,0.82,0.70,4.05,4.42,3.69
500,Zach LaVine,CHI,51,24,27,35.9,13948.0,2.64,1.41,1.23,4.13,4.58,3.73
501,Zeke Nnaji,DEN,43,29,14,13.5,5272.2,1.00,0.53,0.47,4.10,4.24,3.95
502,Ziaire Williams,MEM,28,16,12,17.4,6947.0,1.32,0.65,0.66,4.21,4.37,4.06


In [ ]:
elbow_touches = download_player_tracking(tracking_elbow_touches)
elbow_touches

,PLAYER,TEAM,GP,W,L,MIN,Touches,Elbow Touches,FGM,FGA,...,PTS,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%
0,A.J. Lawson,DAL,12,5,7,4.2,4.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AJ Green,MIL,25,19,6,8.9,10.9,0.1,0.0,0.0,...,0.0,0.0,0.1,100.0,0.0,50.0,0.0,0.0,0.0,0.0
2,AJ Griffin,ATL,54,25,29,20.5,22.1,0.5,0.2,0.3,...,0.4,85.2,0.2,33.3,0.0,22.2,0.0,3.7,0.0,0.0
3,Aaron Gordon,DEN,49,35,14,30.3,42.8,1.8,0.3,0.5,...,0.9,47.8,1.1,57.8,0.2,17.3,0.1,3.3,0.2,10.0
4,Aaron Holiday,ATL,49,26,23,14.6,21.2,0.1,0.0,0.1,...,0.0,28.6,0.1,42.9,0.0,33.3,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,34.2,3.5,1.0,1.4,...,2.1,58.9,1.7,47.9,0.5,28.2,0.3,7.4,0.1,1.8
500,Zach LaVine,CHI,52,24,28,36.0,62.8,0.5,0.1,0.3,...,0.3,64.3,0.1,21.4,0.0,0.0,0.1,14.3,0.0,7.1
501,Zeke Nnaji,DEN,43,29,14,13.5,12.5,1.1,0.2,0.3,...,0.5,41.7,0.8,72.9,0.1,8.6,0.0,0.0,0.0,4.2
502,Ziaire Williams,MEM,28,16,12,17.4,19.0,0.2,0.0,0.1,...,0.2,100.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0


In [ ]:
post_ups = download_player_tracking(tracking_post_ups)
post_ups

,PLAYER,TEAM,GP,W,L,MIN,Touches,Post Ups,FGM,FGA,...,PTS,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%
0,A.J. Lawson,DAL,12,5,7,4.2,4.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AJ Green,MIL,25,19,6,8.9,10.9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AJ Griffin,ATL,53,25,28,20.6,22.2,0.0,0.0,0.0,...,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aaron Gordon,DEN,49,35,14,30.3,42.8,2.3,0.6,1.2,...,1.7,75.7,0.5,21.6,0.1,3.6,0.1,6.3,0.3,13.5
4,Aaron Holiday,ATL,49,26,23,14.6,21.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,45,13,32,20.9,33.8,2.8,0.8,1.4,...,2.2,78.6,0.7,25.4,0.1,4.0,0.2,7.9,0.3,11.1
500,Zach LaVine,CHI,51,24,27,35.9,62.8,0.2,0.0,0.1,...,0.0,25.0,0.1,37.5,0.0,25.0,0.0,12.5,0.0,12.5
501,Zeke Nnaji,DEN,43,29,14,13.5,12.5,0.4,0.2,0.3,...,0.4,94.4,0.1,16.7,0.0,5.6,0.0,0.0,0.0,5.6
502,Ziaire Williams,MEM,28,16,12,17.4,19.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
paint_touches = download_player_tracking(tracking_paint_touches)
paint_touches

,PLAYER,TEAM,GP,W,L,MIN,Touches,Paint Touches,FGM,FGA,...,PTS,PTS%,Pass,Pass%,AST,AST%,TO,TOV%,PF,PF%
0,A.J. Lawson,DAL,12,5,7,4.2,4.1,0.1,0.1,0.1,...,0.2,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AJ Green,MIL,25,19,6,8.9,10.9,0.4,0.1,0.2,...,0.3,88.9,0.1,33.3,0.0,11.1,0.0,11.1,0.0,11.1
2,AJ Griffin,ATL,54,25,29,20.5,22.1,0.6,0.3,0.4,...,0.5,87.9,0.2,33.3,0.1,9.1,0.0,3.0,0.0,3.0
3,Aaron Gordon,DEN,49,35,14,30.3,42.8,7.0,3.6,4.8,...,8.2,118.5,0.9,12.9,0.2,2.9,0.2,3.2,0.7,10.3
4,Aaron Holiday,ATL,49,26,23,14.6,21.2,0.4,0.1,0.2,...,0.2,57.1,0.1,33.3,0.0,0.0,0.0,0.0,0.0,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Zach Collins,SAS,46,13,33,21.1,34.2,4.9,1.7,2.5,...,4.2,85.3,1.4,28.1,0.2,4.5,0.4,7.6,0.3,6.7
500,Zach LaVine,CHI,52,24,28,36.0,62.8,0.9,0.6,0.7,...,1.2,129.2,0.1,10.4,0.0,2.1,0.0,4.2,0.0,2.1
501,Zeke Nnaji,DEN,43,29,14,13.5,12.5,3.0,1.3,2.0,...,3.2,108.6,0.4,13.3,0.0,1.6,0.1,3.9,0.3,10.9
502,Ziaire Williams,MEM,28,16,12,17.4,19.0,1.3,0.7,0.9,...,1.7,137.1,0.0,2.9,0.0,0.0,0.1,8.6,0.2,14.3


In [ ]:
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')

In [ ]:
# save dataframes to csv

catch_shoot.to_csv('data/player/nba_com_playerdata/tracking/catch_shoot_'+ today + '_.csv')
drives.to_csv('data/player/nba_com_playerdata/tracking/drives_'+ today + '_.csv')
defensive_impact.to_csv('data/player/nba_com_playerdata/tracking/defensive_impact_'+ today + '_.csv')
passing.to_csv('data/player/nba_com_playerdata/tracking/passing_'+ today + '_.csv')
touches.to_csv('data/player/nba_com_playerdata/tracking/touches_'+ today + '_.csv')
pull_up_shots.to_csv('data/player/nba_com_playerdata/tracking/pull_up_shots_'+ today + '_.csv')
rebounds.to_csv('data/player/nba_com_playerdata/tracking/rebounds_'+ today + '_.csv')
shooting_efficiency.to_csv('data/player/nba_com_playerdata/tracking/shooting_efficiency_'+ today + '_.csv')
speed_distance.to_csv('data/player/nba_com_playerdata/tracking/speed_distance_'+ today + '_.csv')
elbow_touches.to_csv('data/player/nba_com_playerdata/tracking/elbow_touches_'+ today + '_.csv')
post_ups.to_csv('data/player/nba_com_playerdata/tracking/post_ups_'+ today + '_.csv')
paint_touches.to_csv('data/player/nba_com_playerdata/tracking/paint_touches_'+ today + '_.csv')


## Shooting (same as shot splits?)


In [ ]:
shooting_5ft_url = 'https://www.nba.com/stats/players/shooting'

shooting_5ft = download_player_tracking(shooting_5ft_url)
# get level one column names
level_one = list(shooting_5ft.columns.get_level_values(0))
# move first three columns to the end
level_one = level_one[3:] + level_one[:3]
# get level two column names
level_two = list(shooting_5ft.columns.get_level_values(1))
# combine level one and level two column names
columns = [level_one[i] + '_' + level_two[i] for i in range(len(level_one))]
# set columns
shooting_5ft.columns = columns
# get first column name, rename it to 'Player'
first_column = shooting_5ft.columns[0]
shooting_5ft = shooting_5ft.rename(columns={first_column: 'Player'})
# get second column name, rename it to 'Team'
second_column = shooting_5ft.columns[1]
shooting_5ft = shooting_5ft.rename(columns={second_column: 'Team'})
# get third column name, rename it to 'Age'
third_column = shooting_5ft.columns[2]
shooting_5ft = shooting_5ft.rename(columns={third_column: 'Age'})
# drop the last 3 columns
shooting_5ft = shooting_5ft.drop(shooting_5ft.columns[-3:], axis=1)

shooting_5ft.to_csv('data/player/nba_com_playerdata/shooting/shooting_splits_5ft_'+ today + '_.csv')

shooting_5ft


,Player,Team,Age,Less than 5ft._FGM,Less than 5ft._FGA,Less than 5ft._FG%,5-9 ft._FGM,5-9 ft._FGA,5-9 ft._FG%,10-14 ft._FGM,...,10-14 ft._FG%,15-19 ft._FGM,15-19 ft._FGA,15-19 ft._FG%,20-24 ft._FGM,20-24 ft._FGA,20-24 ft._FG%,25-29 ft._FGM,25-29 ft._FGA,25-29 ft._FG%
0,A.J. Lawson,DAL,22,0.4,0.7,57.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,1.0,22.2,0.3,0.4,75.0
1,AJ Green,MIL,23,0.1,0.2,75.0,0.0,0.0,0.0,0.0,...,33.3,0.2,0.3,66.7,0.3,0.9,33.3,0.8,2.0,42.0
2,AJ Griffin,ATL,19,0.7,1.1,66.7,0.4,0.8,51.2,0.7,...,54.8,0.3,0.6,48.5,0.6,1.6,40.2,0.9,2.3,38.2
3,Aaron Gordon,DEN,27,5.0,7.0,70.8,0.3,0.7,41.2,0.1,...,31.8,0.2,0.5,36.0,0.3,1.0,34.0,0.7,1.5,45.2
4,Aaron Holiday,ATL,26,0.6,1.1,50.9,0.1,0.4,25.0,0.1,...,33.3,0.1,0.2,45.5,0.3,0.6,41.9,0.4,1.0,42.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,25,2.1,3.1,68.2,0.7,1.3,50.0,0.2,...,42.3,0.2,0.4,55.0,0.1,0.4,23.8,0.5,1.2,40.4
501,Zach LaVine,CHI,27,3.6,5.6,65.1,0.5,1.1,40.0,0.4,...,45.5,0.7,1.8,38.9,1.3,3.2,40.0,1.8,5.1,36.4
502,Zeke Nnaji,DEN,22,1.7,2.4,70.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,100,0.1,0.5,19.0,0.2,0.6,36.0
503,Ziaire Williams,MEM,21,1.1,1.6,70.5,0.2,0.4,54.5,0.2,...,42.9,0.1,0.3,42.9,0.5,1.7,27.1,0.3,1.2,20.6


In [ ]:
shooting_by_zone_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone'
shooting_by_zone = download_player_tracking(shooting_by_zone_url)

# get level one column names
level_one = list(shooting_by_zone.columns.get_level_values(0))
# move first three columns to the end
level_one = level_one[3:] + level_one[:3]
# get level two column names
level_two = list(shooting_by_zone.columns.get_level_values(1))
# combine level one and level two column names
columns = [level_one[i] + '_' + level_two[i] for i in range(len(level_one))]
# set columns
shooting_by_zone.columns = columns
# get first column name, rename it to 'Player'
first_column = shooting_by_zone.columns[0]
shooting_by_zone = shooting_by_zone.rename(columns={first_column: 'Player'})
# get second column name, rename it to 'Team'
second_column = shooting_by_zone.columns[1]
shooting_by_zone = shooting_by_zone.rename(columns={second_column: 'Team'})
# get third column name, rename it to 'Age'
third_column = shooting_by_zone.columns[2]
shooting_by_zone = shooting_by_zone.rename(columns={third_column: 'Age'})
# drop the last 3 columns
shooting_by_zone = shooting_by_zone.drop(shooting_by_zone.columns[-3:], axis=1)

shooting_by_zone.to_csv('data/player/nba_com_playerdata/shooting/shooting_splits_by_zone_'+ today + '_.csv')

shooting_by_zone

,Player,Team,Age,Restricted Area_FGM,Restricted Area_FGA,Restricted Area_FG%,In The Paint (Non-RA)_FGM,In The Paint (Non-RA)_FGA,In The Paint (Non-RA)_FG%,Mid-Range_FGM,...,Left Corner 3._FG%,Right Corner 3._FGM,Right Corner 3._FGA,Right Corner 3._FG%,Corner 3_FGM,Corner 3_FGA,Corner 3_FG%,Above the Break 3._FGM,Above the Break 3._FGA,Above the Break 3._FG%
0,A.J. Lawson,DAL,22,0.3,0.6,50.0,0.1,0.1,100,0.0,...,0.0,0.3,0.8,33.3,0.1,1.0,14.3,0.3,0.5,66.7
1,AJ Green,MIL,23,0.1,0.2,75.0,0.1,0.1,100,0.2,...,16.7,0.3,0.5,50.0,0.3,0.8,35.7,0.9,2.2,41.1
2,AJ Griffin,ATL,19,0.6,0.9,72.3,1.0,1.9,50.5,0.5,...,44.4,0.1,0.3,31.3,0.5,1.1,41.0,1.1,2.7,39.3
3,Aaron Gordon,DEN,27,4.9,6.7,72.6,0.5,1.2,39.7,0.2,...,25.0,0.1,0.4,30.0,0.2,0.7,27.8,0.8,1.8,46.0
4,Aaron Holiday,ATL,26,0.6,1.0,56.3,0.3,0.9,30.2,0.1,...,42.9,0.0,0.2,12.5,0.1,0.3,26.7,0.6,1.2,45.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,25,1.9,2.8,69.5,1.0,1.8,53.5,0.4,...,0.0,0.0,0.1,50.0,0.0,0.1,40.0,0.5,1.5,35.7
501,Zach LaVine,CHI,27,3.5,5.2,66.2,0.8,1.7,44.4,1.4,...,42.9,0.2,0.5,39.3,0.4,1.1,41.1,2.3,6.3,36.4
502,Zeke Nnaji,DEN,22,1.6,2.3,72.0,0.0,0.1,20.0,0.0,...,14.3,0.1,0.3,30.0,0.1,0.5,23.5,0.2,0.7,32.1
503,Ziaire Williams,MEM,21,1.1,1.5,72.1,0.4,0.8,52.4,0.2,...,18.5,0.3,0.5,46.2,0.4,1.5,27.5,0.3,1.4,20.5


## Shot Dashboard

In [ ]:
shot_dash_general_url = 'https://www.nba.com/stats/players/shots-general'
shot_dash_general = download_player_tracking(shot_dash_general_url)

# drop top level of column names
shot_dash_general.columns = shot_dash_general.columns.droplevel(0)

# drop unnamed cols
unnamed = [col for col in shot_dash_general.columns if 'Unnamed' in col]
shot_dash_general = shot_dash_general.drop(unnamed, axis=1)

shot_dash_general.to_csv('data/player/nba_com_playerdata/shooting/shot_dash_general_'+ today + '_.csv')

shot_dash_general


,PLAYER,TEAM,AGE,GP,G,FREQ,FGM,FGA,FG%,EFG%,2FG FREQ,2FGM,2FGA,2FG%,3FG FREQ,3PM,3PA,3P%
0,LeBron James,LAL,38,44,44,100,11.6,22.8,50.9,55.7,69.3,9.4,15.8,59.5,30.7,2.2,7.0,31.4
1,Luka Doncic,DAL,23,49,49,100,11.2,22.2,50.5,56.6,64.8,8.5,14.4,58.9,35.2,2.7,7.8,34.8
2,Jayson Tatum,BOS,24,54,54,100,9.8,21.4,46.1,53.8,56.5,6.6,12.1,54.4,43.5,3.3,9.3,35.3
3,Giannis Antetokounmpo,MIL,28,45,45,100,11.5,21.3,54.2,56.1,85.7,10.7,18.2,58.7,14.3,0.8,3.0,27.0
4,Ja Morant,MEM,23,47,47,100,9.7,20.7,47.1,51.2,74.8,8.0,15.5,51.9,25.2,1.7,5.2,32.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Andre Iguodala,GSW,39,2,1,100,0.5,0.5,100.0,100.0,100.0,0.5,0.5,100,0.0,0.0,0.0,-
496,John Butler Jr.,POR,20,8,3,100,0.1,0.5,25.0,25.0,25.0,0.1,0.1,100,75.0,0.0,0.4,0.0
497,Ryan Arcidiacono,POR,28,12,5,100,0.1,0.5,16.7,25.0,33.3,0.0,0.2,0.0,66.7,0.1,0.3,24.2
498,Braxton Key,DET,25,3,1,100,0.3,0.3,100.0,100.0,100.0,0.3,0.3,100,0.0,0.0,0.0,-


Catch and Shoot

In [ ]:
catch_and_shoot_url = 'https://www.nba.com/stats/players/shots-general?GeneralRange=Catch+and+Shoot'
catch_and_shoot = download_player_tracking(catch_and_shoot_url)

# drop top level of column names
catch_and_shoot.columns = catch_and_shoot.columns.droplevel(0)

# drop unnamed cols
unnamed = [col for col in catch_and_shoot.columns if 'Unnamed' in col]
catch_and_shoot = catch_and_shoot.drop(unnamed, axis=1)

catch_and_shoot.to_csv('data/player/nba_com_playerdata/shooting/catch_and_shoot_'+ today + '_.csv')

catch_and_shoot

,PLAYER,TEAM,AGE,GP,G,FREQ,FGM,FGA,FG%,EFG%,2FG FREQ,2FGM,2FGA,2FG%,3FG FREQ,3PM,3PA,3P%
0,Klay Thompson,GSW,33,44,44,48.5,3.6,9.0,39.5,57.3,5.5,0.4,1.0,35.3,43.0,3.2,8.0,40.0
1,Lauri Markkanen,UTA,25,54,54,41.0,2.9,6.7,42.7,63.2,1.2,0.1,0.2,55.0,39.7,2.8,6.5,42.3
2,Nikola Vucevic,CHI,32,56,56,47.6,2.7,6.6,40.1,52.6,14.5,1.0,2.0,49.5,33.2,1.7,4.6,35.9
3,Buddy Hield,IND,30,59,59,47.5,2.7,6.5,41.8,62.5,0.5,0.0,0.1,28.6,47.0,2.7,6.4,41.9
4,Tim Hardaway Jr.,DAL,30,54,54,52.2,2.3,6.4,36.2,53.5,1.8,0.1,0.2,50.0,50.4,2.2,6.2,35.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,Walker Kessler,UTA,21,56,2,0.7,0.0,0.0,50.0,50.0,0.7,0.0,0.0,50.0,0.0,0.0,0.0,-
484,Udoka Azubuike,UTA,23,23,1,4.2,0.0,0.0,0.0,0.0,4.2,0.0,0.0,0.0,0.0,0.0,0.0,-
485,DeAndre Jordan,DEN,34,34,1,1.0,0.0,0.0,100.0,150.0,0.0,0.0,0.0,-,1.0,0.0,0.0,100
486,Bruno Fernando,ATL,24,31,1,1.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,1.1,0.0,0.0,0.0


Pullups

In [ ]:
pullups_url = 'https://www.nba.com/stats/players/shots-general?GeneralRange=Pullups'
pullups = download_player_tracking(pullups_url)

# drop top level of column names
pullups.columns = pullups.columns.droplevel(0)

# drop unnamed cols
unnamed = [col for col in pullups.columns if 'Unnamed' in col]
pullups = pullups.drop(unnamed, axis=1)

pullups.to_csv('data/player/nba_com_playerdata/shooting/pullups_'+ today + '_.csv')

pullups

,PLAYER,TEAM,AGE,GP,G,FREQ,FGM,FGA,FG%,EFG%,2FG FREQ,2FGM,2FGA,2FG%,3FG FREQ,3PM,3PA,3P%
0,Luka Doncic,DAL,23,48,48,54.0,4.8,12.0,40.0,49.9,22.9,2.4,5.1,47.5,31.1,2.4,6.9,34.6
1,Devin Booker,PHX,26,31,30,56.7,4.9,11.4,42.8,49.0,35.9,3.5,7.3,47.9,20.8,1.4,4.2,33.9
2,Trae Young,ATL,24,50,50,54.0,4.0,10.7,37.8,45.5,27.4,2.4,5.4,43.7,26.5,1.7,5.3,31.6
3,DeMar DeRozan,CHI,33,51,51,58.2,5.0,10.5,47.6,48.0,56.7,4.9,10.2,47.8,1.5,0.1,0.3,37.0
4,Kevin Durant,PHX,34,39,39,55.1,5.6,10.3,54.7,59.0,42.8,4.8,8.0,59.6,12.3,0.9,2.3,37.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Zeke Nnaji,DEN,22,44,1,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,0.7,0.0,0.0,0.0
457,Steven Adams,MEM,29,42,1,0.4,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,-
458,Dwight Powell,DAL,31,53,1,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,-
459,Usman Garuba,HOU,20,49,1,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,0.8,0.0,0.0,0.0


Opponent Shooting

In [ ]:

opp_shooting_5ft = 'https://www.nba.com/stats/players/opponent-shooting'
opp_shooting_5ft = download_player_tracking(opp_shooting_5ft)

# get level one column names
level_one = list(opp_shooting_5ft.columns.get_level_values(0))
# move first three columns to the end
level_one = level_one[3:] + level_one[:3]
# get level two column names
level_two = list(opp_shooting_5ft.columns.get_level_values(1))
# combine level one and level two column names
columns = [level_one[i] + '_' + level_two[i] for i in range(len(level_one))]
# set columns
opp_shooting_5ft.columns = columns
# get first column name, rename it to 'Player'
first_column = opp_shooting_5ft.columns[0]
opp_shooting_5ft = opp_shooting_5ft.rename(columns={first_column: 'Player'})
# get second column name, rename it to 'Team'
second_column = opp_shooting_5ft.columns[1]
opp_shooting_5ft = opp_shooting_5ft.rename(columns={second_column: 'Team'})
# get third column name, rename it to 'Age'
third_column = opp_shooting_5ft.columns[2]
opp_shooting_5ft = opp_shooting_5ft.rename(columns={third_column: 'Age'})
# drop the last 3 columns
opp_shooting_5ft = opp_shooting_5ft.drop(opp_shooting_5ft.columns[-3:], axis=1)

opp_shooting_5ft.to_csv('data/player/nba_com_playerdata/shooting/opp_shooting_5ft_'+ today + '_.csv')

opp_shooting_5ft


,Player,Team,Age,Less than 5ft._FGM,Less than 5ft._FGA,Less than 5ft._FG%,5-9 ft._FGM,5-9 ft._FGA,5-9 ft._FG%,10-14 ft._FGM,...,10-14 ft._FG%,15-19 ft._FGM,15-19 ft._FGA,15-19 ft._FG%,20-24 ft._FGM,20-24 ft._FGA,20-24 ft._FG%,25-29 ft._FGM,25-29 ft._FGA,25-29 ft._FG%
0,A.J. Lawson,DAL,22,2.2,2.8,78.6,0.6,1.0,55.6,0.8,...,54.5,0.3,0.4,66.7,0.4,1.7,26.7,0.6,1.8,33.3
1,AJ Green,MIL,23,4.0,6.1,66.4,1.0,2.3,43.8,1.0,...,46.5,0.7,1.5,44.4,0.6,1.9,32.4,1.8,4.8,36.7
2,AJ Griffin,ATL,19,8.7,13.7,63.4,1.6,3.7,42.0,1.4,...,40.8,0.8,1.8,42.9,2.5,6.3,39.7,2.8,8.7,31.8
3,Aaron Gordon,DEN,27,12.7,19.0,66.7,2.6,6.7,39.0,1.9,...,44.0,1.3,3.3,40.5,3.1,8.6,36.8,4.3,12.8,34.0
4,Aaron Holiday,ATL,26,6.7,10.4,64.9,1.0,2.6,38.3,0.8,...,36.2,0.6,1.5,40.3,1.6,4.1,39.6,2.1,6.1,33.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,25,9.2,14.6,63.0,2.1,4.6,45.1,1.5,...,43.4,1.7,3.0,55.2,2.0,5.0,39.8,3.1,8.8,34.8
501,Zach LaVine,CHI,27,14.9,22.4,66.6,2.8,6.8,40.4,1.9,...,42.6,1.6,3.8,41.2,3.8,11.1,34.8,5.9,16.8,35.1
502,Zeke Nnaji,DEN,22,5.8,8.8,65.5,1.4,3.1,44.9,0.7,...,40.3,0.6,1.3,42.6,1.3,3.9,33.5,1.5,5.6,27.5
503,Ziaire Williams,MEM,21,6.3,10.5,59.7,1.5,3.3,45.2,1.0,...,38.7,1.0,2.0,50.0,1.9,5.5,34.4,3.1,8.8,35.5


Opp Shooting by Zone

In [ ]:
opp_shooting_by_zone_url = 'https://www.nba.com/stats/players/opponent-shooting?DistanceRange=By+Zone'
opp_shooting_by_zone = download_player_tracking(opp_shooting_by_zone_url)

# get level one column names
level_one = list(opp_shooting_by_zone.columns.get_level_values(0))
# move first three columns to the end
level_one = level_one[3:] + level_one[:3]
# get level two column names
level_two = list(opp_shooting_by_zone.columns.get_level_values(1))
# combine level one and level two column names
columns = [level_one[i] + '_' + level_two[i] for i in range(len(level_one))]
# set columns
opp_shooting_by_zone.columns = columns
# get first column name, rename it to 'Player'
first_column = opp_shooting_by_zone.columns[0]
opp_shooting_by_zone = opp_shooting_by_zone.rename(columns={first_column: 'Player'})
# get second column name, rename it to 'Team'
second_column = opp_shooting_by_zone.columns[1]
opp_shooting_by_zone = opp_shooting_by_zone.rename(columns={second_column: 'Team'})
# get third column name, rename it to 'Age'
third_column = opp_shooting_by_zone.columns[2]
opp_shooting_by_zone = opp_shooting_by_zone.rename(columns={third_column: 'Age'})
# drop the last 3 columns
opp_shooting_by_zone = opp_shooting_by_zone.drop(opp_shooting_by_zone.columns[-3:], axis=1)

opp_shooting_by_zone.to_csv('data/player/nba_com_playerdata/shooting/opp_shooting_by_zone_'+ today + '_.csv')

opp_shooting_by_zone

,Player,Team,Age,Restricted Area_FGM,Restricted Area_FGA,Restricted Area_FG%,In The Paint (Non-RA)_FGM,In The Paint (Non-RA)_FGA,In The Paint (Non-RA)_FG%,Mid-Range_FGM,...,Left Corner 3._FG%,Right Corner 3._FGM,Right Corner 3._FGA,Right Corner 3._FG%,Corner 3_FGM,Corner 3_FGA,Corner 3_FG%,Above the Break 3._FGM,Above the Break 3._FGA,Above the Break 3._FG%
0,A.J. Lawson,DAL,22,2.0,2.5,80.0,0.8,1.7,46.7,1.0,...,25.0,0.3,0.3,100,0.3,0.7,40.0,0.7,2.2,30.8
1,AJ Green,MIL,23,3.7,5.4,68.5,1.9,4.1,46.8,0.9,...,23.1,0.3,0.8,33.3,0.4,1.5,28.0,1.9,5.2,36.9
2,AJ Griffin,ATL,19,8.3,12.7,65.3,2.6,6.4,40.4,1.8,...,45.8,0.8,2.0,37.9,1.7,4.0,42.3,3.4,10.5,32.5
3,Aaron Gordon,DEN,27,11.6,16.6,69.7,4.8,11.4,41.9,2.5,...,35.2,1.1,2.5,42.5,2.2,5.8,38.2,5.1,15.0,33.9
4,Aaron Holiday,ATL,26,6.5,9.6,68.2,1.6,4.5,35.5,1.0,...,51.4,0.6,1.4,38.5,1.2,2.8,45.2,2.3,7.1,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Zach Collins,SAS,25,8.6,13.2,64.8,3.6,7.6,47.5,2.6,...,45.1,0.4,1.6,27.5,1.1,3.0,36.4,3.8,10.6,35.5
501,Zach LaVine,CHI,27,14.2,20.9,68.1,4.2,10.4,40.6,3.0,...,38.3,1.0,3.5,29.3,2.3,6.9,33.7,7.2,20.4,35.5
502,Zeke Nnaji,DEN,22,5.4,7.9,67.8,2.1,4.7,44.8,0.9,...,35.8,0.5,1.4,40.5,1.0,2.6,37.9,1.8,6.6,27.1
503,Ziaire Williams,MEM,21,6.0,9.5,62.8,2.3,5.5,41.3,1.9,...,31.7,0.5,1.3,38.7,1.0,2.8,34.7,3.9,11.2,34.4


Add: Player Offensive Playtype

In [ ]:
pt_transition_url = 'https://www.nba.com/stats/players/transition'
pt_isolation_url = 'https://www.nba.com/stats/players/isolation'
pt_pr_ball_handler_url = 'https://www.nba.com/stats/players/ball-handler'
pt_pr_roll_man_url = 'https://www.nba.com/stats/players/roll-man'
pt_post_up_url = 'https://www.nba.com/stats/players/playtype-post-up'
pt_spot_up_url = 'https://www.nba.com/stats/players/spot-up'
pt_hand_off_url = 'https://www.nba.com/stats/players/hand-off'
pt_cut_url = 'https://www.nba.com/stats/players/cut'
pt_off_screen_url = 'https://www.nba.com/stats/players/off-screen'
pt_putbacks_url = 'https://www.nba.com/stats/players/putbacks'

In [ ]:
def download_player_playtype(url):
    try:
        driver.get(url)
        # wait 2 seconds
        time.sleep(8)
        # select all pages
        all_pages_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]'
        all_pages = driver.find_element_by_xpath(all_pages_xpath)
        all_pages.click()
        # wait 5 seconds
        time.sleep(5)
        # find table
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find_all('table')
        # read table with pandas
        df = pd.read_html(str(table))[0]
        return df
    except:
        print('That didnt work')
        pass

In [ ]:
driver = webdriver.Chrome()

In [ ]:
pt_transition = download_player_playtype(pt_transition_url)
pt_isolation = download_player_playtype(pt_isolation_url)
pt_pr_ball_handler = download_player_playtype(pt_pr_ball_handler_url)
pt_pr_roll_man = download_player_playtype(pt_pr_roll_man_url)
pt_post_up = download_player_playtype(pt_post_up_url)
pt_spot_up = download_player_playtype(pt_spot_up_url)
pt_hand_off = download_player_playtype(pt_hand_off_url)
pt_cut = download_player_playtype(pt_cut_url)
pt_off_screen = download_player_playtype(pt_off_screen_url)
pt_putbacks = download_player_playtype(pt_putbacks_url)

In [ ]:
# save each to data/player/nba_com_playerdata/playtypes/
pt_transition.to_csv('data/player/nba_com_playerdata/playtypes/pt_transition_'+ today + '_.csv')
pt_isolation.to_csv('data/player/nba_com_playerdata/playtypes/pt_isolation_'+ today + '_.csv')
pt_pr_ball_handler.to_csv('data/player/nba_com_playerdata/playtypes/pt_pr_ball_handler_'+ today + '_.csv')
pt_pr_roll_man.to_csv('data/player/nba_com_playerdata/playtypes/pt_pr_roll_man_'+ today + '_.csv')
pt_post_up.to_csv('data/player/nba_com_playerdata/playtypes/pt_post_up_'+ today + '_.csv')
pt_spot_up.to_csv('data/player/nba_com_playerdata/playtypes/pt_spot_up_'+ today + '_.csv')
pt_hand_off.to_csv('data/player/nba_com_playerdata/playtypes/pt_hand_off_'+ today + '_.csv')
pt_cut.to_csv('data/player/nba_com_playerdata/playtypes/pt_cut_'+ today + '_.csv')
pt_off_screen.to_csv('data/player/nba_com_playerdata/playtypes/pt_off_screen_'+ today + '_.csv')
pt_putbacks.to_csv('data/player/nba_com_playerdata/playtypes/pt_putbacks_'+ today + '_.csv')